In [1]:
#importing what is needed
from datetime import date
import time
import pandas as pd
import requests
import json5
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

# hide warnings
import warnings
warnings.filterwarnings('ignore')


In [2]:
opt = Options();
opt.add_argument("headless");
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opt)
driver.implicitly_wait(0.5)



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
There is no [win32] chromedriver for browser  in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Anned\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [3]:
url_base = 'https://www.indeed.com/jobs?q&l=remote&fromage=7&limit=50&sort=date'

In [4]:
#nned to fix this
job_count = driver.find_element(By.ID, "searchCountPages")
job_count = job_count.text
job_count = job_count[job_count.find('of')+3:job_count.find('jobs')-1]
loc = job_count.find(',')
while loc != -1:
    job_count = job_count.replace(',','')
    loc = job_count.find(',')
job_count = int(job_count)
driver.quit()
jobs = []
job_count

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="searchCountPages"]"}
  (Session info: headless chrome=98.0.4758.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x003F69A3+2582947]
	Ordinal0 [0x0038A6D1+2139857]
	Ordinal0 [0x00283A98+1063576]
	Ordinal0 [0x002AFF3E+1244990]
	Ordinal0 [0x002B013B+1245499]
	Ordinal0 [0x002D9F52+1417042]
	Ordinal0 [0x002C8594+1344916]
	Ordinal0 [0x002D834A+1409866]
	Ordinal0 [0x002C8366+1344358]
	Ordinal0 [0x002A5176+1200502]
	Ordinal0 [0x002A6066+1204326]
	GetHandleVerifier [0x0059BE02+1675858]
	GetHandleVerifier [0x0065036C+2414524]
	GetHandleVerifier [0x0048BB01+560977]
	GetHandleVerifier [0x0048A8D3+556323]
	Ordinal0 [0x0039020E+2163214]
	Ordinal0 [0x00395078+2183288]
	Ordinal0 [0x003951C0+2183616]
	Ordinal0 [0x0039EE1C+2223644]
	BaseThreadInitThunk [0x75F96739+25]
	RtlGetFullPathName_UEx [0x771A8E7F+1215]
	RtlGetFullPathName_UEx [0x771A8E4D+1165]
